In [1]:
import multiprocessing as mp
import pickle

In [2]:
import numpy as np

from scipy.integrate import solve_ivp

In [3]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [4]:
from cybernetics.controls.piecewise_linear_controller import PiecewiseLinearController as PWL_Controller

from osol.extremum.algorithms.statistical_anti_gradient_random_search import StatisticalAntiGradientRandomSearch as SAG_RS

In [5]:
from models import Model_1

In [6]:
grid_size = 11
grid = np.linspace(0.0, 1.0, num=grid_size)
search_area = [(-100, 100) for _ in range(grid_size)]

sag_rs_radius = 1.0
sag_rs_number_of_samples = 10

In [7]:
def task_1(x0, work_time=(60 * 10)):
    def f(control_settings):
        controller = PWL_Controller(grid, control_settings)
        model = Model_1(control=lambda t, _: controller.generate(t))
        sol = solve_ivp(
            model, [0, 1], [x0, 0.0], 
            t_eval=np.linspace(0, 1, num=101), 
            method="RK45")
        return model.efficiency(sol)[0]
    
    tool = SAG_RS(
        radius=sag_rs_radius, 
        number_of_samples=sag_rs_number_of_samples)
    
    tool.initialize(x=np.array(([0] * grid_size)), f=f)
    answer = tool.optimize_max_runtime(f, search_area, max_seconds=work_time)
    
    controller = PWL_Controller(grid, answer)
    model = Model_1(control=lambda t, _: controller.generate(t))
    sol = solve_ivp(model, [0, 1], [x0, 0.0], t_eval=np.linspace(0, 1, num=101), method="RK45")
    I, control = model.efficiency(sol)
    
    return x0, answer, model, sol, I, control

In [ ]:
pool = mp.Pool(12)
results = pool.map(task_1, np.linspace(-25, 25, 51))
pool.close()

In [ ]:
with open("results.pkl", "wb") as f:
    pickle.dump(results, f)